## 1. Dataset Preparation
### 1.1 Loading Datasets

**Document Type** = Article, Article in Press, Review, Book Chapter, Letter, Note

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import scipy.stats as stats
import pickle

from itertools import chain, combinations
from scipy.cluster.hierarchy import dendrogram
from unidecode import unidecode
from collections import Counter
from operator import itemgetter

In [ ]:
authorsPath = "data/authors.xlsx"
epidemiologyPath = "data/epidemiology.xlsx"
immunologyPath = "data/immunology.xlsx"
infectiveDiseasesPath = "data/infectious_diseases.xlsx"
microbiologyPath = "data/microbiology.xlsx"

authorsData = pd.read_excel(authorsPath)

epidemiologyData = pd.read_excel(epidemiologyPath)
immunologyData = pd.read_excel(immunologyPath)
infectiveDiseasesData = pd.read_excel(infectiveDiseasesPath)
microbiologyData = pd.read_excel(microbiologyPath)

documentTypeSet = {"Article", "Article in Press", "Review", "Book Chapter", "Letter", "Note"}

duplicatesSubset = ['Authors', 'Title', 'Year', 'Source title', 'Document Type']

print(f"Autori - Rows: {authorsData.shape[0]} - Cols: {authorsData.shape[1]}")
print(f"Epidemiology - Rows: {epidemiologyData.shape[0]} - Cols: {epidemiologyData.shape[1]}")
print(f"Immunology - Rows: {immunologyData.shape[0]} - Cols: {immunologyData.shape[1]}")
print(f"Infectious diseases - Rows: {infectiveDiseasesData.shape[0]} - Cols: {infectiveDiseasesData.shape[1]}")
print(f"Microbiology - Rows: {microbiologyData.shape[0]} - Cols: {microbiologyData.shape[1]}")

# Global helper functions

def convertSerbianCharsToAscii(df, columns):
    serbianLatinCharsMap = {
        'ć': 'c',
        'č': 'c',
        'đ': 'dj',
        'š': 's',
        'ž': 'z',
        
        'Ć': 'C',
        'Č': 'C',
        'Đ': 'Dj',
        'Š': 'S',
        'Ž': 'Z',
    }
    
    for column in columns:
        df[column] = df[column].apply(lambda x: unidecode(x,  errors='strict'))


### 1.2 Dataset Preprocessing with Authors
Some authors were identified whose **Number of Publications** and **H-index** are null or unknown.  

**Number of Publications** – will be set to 0 for now, and we will attempt to fill it in later from the corresponding department dataset.  
**H-index** – will be set to 0.  
**Short Name** – a new column has been added representing the author's short name.


In [ ]:
authorsData.fillna(0, inplace=True)
authorsData['H index'] = authorsData['H index'].replace('?', 0)
authorsData = authorsData.astype({'H index': int, 'Number of rows': int})

convertSerbianCharsToAscii(authorsData, ['Name', 'Surname', 'Department'])

authorsData.insert(0,'id_author',0)
authorsData['id_author'] = authorsData.reset_index().index + 1

authorsData['Name'] = authorsData['Name'].apply(lambda x: x.strip())
authorsData['Surname'] = authorsData['Surname'].apply(lambda x: x.strip().replace('-', ' '))


authorsData['Short name'] = authorsData['Surname'] + ' ' + authorsData['Name'].str[0] + '.'


# duplicatedAuthors = authorsData.duplicated(subset=['Name', 'Surname'])
# print(duplicatedAuthors)

print(authorsData.dtypes)

# print(authorsData['Short name'])

print(authorsData['Short name'].is_unique)

# print(authorsData)

authorsData.head(-1)

authorsData.to_excel("data/authorsData.xlsx") 

## 1.3 Preparation of Publication Datasets
* Serbian Latin characters were removed  
* A new column 'Department name' was added  
* The following columns were removed as irrelevant: 'Art. No.', 'Page start', 'Page end', 'Page count', 'Link', 'Source', 'Volume', 'Issue'  
* Publications were filtered by 'Document Type' ("Article", "Article in Press", "Review", "Book Chapter", "Letter", "Note")  
* Duplicates were removed – A publication is considered a duplicate if the columns 'Authors', 'Title', 'Year', and 'Source title' are identical.

### 1.3.1 Preparation of the Dataset for Epidemiology


In [ ]:
epidemiologyDataFiltered = epidemiologyData.copy(deep=True)

# Replacing Serbian Latin specific chars with Ascii chars

convertSerbianCharsToAscii(epidemiologyDataFiltered, ['Author', 'Authors', 'Title', 'Source title'])

# Adding 'Department name' column

epidemiologyDataFiltered['Research area'] = 'Epidemiology'

# Removing unnecessary charactes from 'Author' and 'Authors' columns

epidemiologyDataFiltered['Author'] = epidemiologyDataFiltered['Author'].apply(lambda x: x.strip().replace('-', ' '))
epidemiologyDataFiltered['Authors'] = epidemiologyDataFiltered['Authors'].apply(lambda x: x.strip().replace('-', ' '))

# Drop unnecessary columns
epidemiologyDataFiltered.drop(columns=['Art. No.'], inplace=True)
epidemiologyDataFiltered.drop(columns=['Page start'], inplace=True)
epidemiologyDataFiltered.drop(columns=['Page end'], inplace=True)
epidemiologyDataFiltered.drop(columns=['Page count'], inplace=True)
epidemiologyDataFiltered.drop(columns=['Link'], inplace=True)
epidemiologyDataFiltered.drop(columns=['Source'], inplace=True)
epidemiologyDataFiltered.drop(columns=['Volume'], inplace=True)
epidemiologyDataFiltered.drop(columns=['Issue'], inplace=True)

# Replacing NaN Values

epidemiologyDataFiltered['Cited by'] = epidemiologyDataFiltered['Cited by'].fillna(0)

# Fixing types
epidemiologyDataFiltered = epidemiologyDataFiltered.astype(
    {
        'Year': int, 
        'Cited by': int
    }
)
print(epidemiologyDataFiltered.dtypes)

# Filtering rows by 'Documnet Type'

epidemiologyDataFiltered = epidemiologyDataFiltered[epidemiologyDataFiltered['Document Type'].isin(documentTypeSet)]

# Checking duplcates and dropping them
epidemiologyDataCountOld = epidemiologyDataFiltered.shape[0]

epidemiologyDataFiltered = epidemiologyDataFiltered.drop_duplicates(
    subset=duplicatesSubset,
    keep="first",
    inplace=False
)

epidemiologyDataCountNew = epidemiologyDataFiltered.shape[0]

print(f"Number of rows before deduplication '{epidemiologyDataCountOld}'")
print(f"Number of rows after deduplication '{epidemiologyDataCountNew}'")
print(f"Total removed'{epidemiologyDataCountOld - epidemiologyDataCountNew}' scientific papers")

epidemiologyDataFiltered.head(20)

### 1.3.2 Preparation of Dataset for Immunology

In [ ]:
immunologyDataFiltered = immunologyData.copy(deep=True)

# Replacing Serbian Latin specific chars with Ascii chars

convertSerbianCharsToAscii(immunologyDataFiltered, ['Author', 'Authors', 'Title', 'Source title'])

# Adding 'Department name' column

immunologyDataFiltered['Research area'] = 'Immunology'

# Removing unnecessary charactes from 'Author' and 'Authors' columns

immunologyDataFiltered['Author'] = immunologyDataFiltered['Author'].apply(lambda x: x.strip().replace('-', ' '))
immunologyDataFiltered['Authors'] = immunologyDataFiltered['Authors'].apply(lambda x: x.strip().replace('-', ' '))

# Drop unnecessary columns
immunologyDataFiltered.drop(columns=['Art. No.'], inplace=True)
immunologyDataFiltered.drop(columns=['Page start'], inplace=True)
immunologyDataFiltered.drop(columns=['Page end'], inplace=True)
immunologyDataFiltered.drop(columns=['Page count'], inplace=True)
immunologyDataFiltered.drop(columns=['Link'], inplace=True)
immunologyDataFiltered.drop(columns=['Source'], inplace=True)
immunologyDataFiltered.drop(columns=['Volume'], inplace=True)
immunologyDataFiltered.drop(columns=['Issue'], inplace=True)


# Replacing NaN Values

immunologyDataFiltered['Cited by'] = immunologyDataFiltered['Cited by'].fillna(0)

# Fixing types
immunologyDataFiltered = immunologyDataFiltered.astype(
    {
        'Year': int, 
        'Cited by': int
    }
)
print(immunologyDataFiltered.dtypes)

# Filtering rows by 'Documnet Type'

immunologyDataFiltered = immunologyDataFiltered[immunologyDataFiltered['Document Type'].isin(documentTypeSet)]


# Checking duplcates and dropping them
immunologyDataCountOld = immunologyDataFiltered.shape[0]

immunologyDataFiltered = immunologyDataFiltered.drop_duplicates(
    subset=duplicatesSubset,
    keep="first",
    inplace=False
)

immunologyDataCountNew = immunologyDataFiltered.shape[0]

print(f"Number of rows before deduplication '{immunologyDataCountOld}'")
print(f"Number of rows after deduplication '{immunologyDataCountNew}'")
print(f"Total removed '{immunologyDataCountOld - immunologyDataCountNew}' scientific papaers")
print(f"Title unique: {immunologyDataFiltered['Title'].is_unique}")

immunologyDataFiltered.head(20)

### 1.3.3 Priprema Dataset-a za Infektivne bolesti

In [ ]:
infectiveDiseasesDataFiltered = infectiveDiseasesData.copy(deep=True)

# Replacing Serbian Latin specific chars with Ascii chars
convertSerbianCharsToAscii(infectiveDiseasesDataFiltered, ['Author', 'Authors', 'Title', 'Source title'])

# Adding 'Department name' column
infectiveDiseasesDataFiltered['Research area'] = 'Infective Diseases'

# Removing unnecessary charactes from 'Author' and 'Authors' columns
infectiveDiseasesDataFiltered['Author'] = infectiveDiseasesDataFiltered['Author'].apply(lambda x: x.strip().replace('-', ' '))
infectiveDiseasesDataFiltered['Authors'] = infectiveDiseasesDataFiltered['Authors'].apply(lambda x: x.strip().replace('-', ' '))

# Drop unnecessary columns
infectiveDiseasesDataFiltered.drop(columns=['Art. No.'], inplace=True)
infectiveDiseasesDataFiltered.drop(columns=['Page start'], inplace=True)
infectiveDiseasesDataFiltered.drop(columns=['Page end'], inplace=True)
infectiveDiseasesDataFiltered.drop(columns=['Page count'], inplace=True)
infectiveDiseasesDataFiltered.drop(columns=['Link'], inplace=True)
infectiveDiseasesDataFiltered.drop(columns=['Source'], inplace=True)
infectiveDiseasesDataFiltered.drop(columns=['Volume'], inplace=True)
infectiveDiseasesDataFiltered.drop(columns=['Issue'], inplace=True)


# Replacing NaN Values
infectiveDiseasesDataFiltered['Cited by'] = infectiveDiseasesDataFiltered['Cited by'].fillna(0)

# Fixing types
infectiveDiseasesDataFiltered = infectiveDiseasesDataFiltered.astype(
    {
        'Year': int, 
        'Cited by': int
    }
)
print(infectiveDiseasesDataFiltered.dtypes)

# Filtering rows by 'Documnet Type'
infectiveDiseasesDataFiltered = infectiveDiseasesDataFiltered[infectiveDiseasesDataFiltered['Document Type'].isin(documentTypeSet)]


# Checking duplcates and dropping them
infectiveDiseasesDataCountOld = infectiveDiseasesDataFiltered.shape[0]

infectiveDiseasesDataFiltered = infectiveDiseasesDataFiltered.drop_duplicates(
    subset=duplicatesSubset,
    keep="first",
    inplace=False
)

infectiveDiseasesDataCountNew = infectiveDiseasesDataFiltered.shape[0]

print(f"Number of rows before deduplication '{infectiveDiseasesDataCountOld}'")
print(f"Number of rows after deduplication '{infectiveDiseasesDataCountNew}'")
print(f"Total removed '{infectiveDiseasesDataCountOld - infectiveDiseasesDataCountNew}' scientific papaers")

infectiveDiseasesDataFiltered.head(20)

### 1.3.4 Preparation of Dataset for Microbiology

In [ ]:
microbiologyDataFiltered = microbiologyData.copy(deep=True)

# Replacing Serbian Latin specific chars with Ascii chars
convertSerbianCharsToAscii(microbiologyDataFiltered, ['Author', 'Authors', 'Title', 'Source title'])

# Adding 'Department name' column
microbiologyDataFiltered['Research area'] = 'Microbiology'

# Removing unnecessary charactes from 'Author' and 'Authors' columns
microbiologyDataFiltered['Author'] = microbiologyDataFiltered['Author'].apply(lambda x: x.strip().replace('-', ' '))
microbiologyDataFiltered['Authors'] = microbiologyDataFiltered['Authors'].apply(lambda x: x.strip().replace('-', ' '))

# Drop unnecessary columns
microbiologyDataFiltered.drop(columns=['Art. No.'], inplace=True)
microbiologyDataFiltered.drop(columns=['Page start'], inplace=True)
microbiologyDataFiltered.drop(columns=['Page end'], inplace=True)
microbiologyDataFiltered.drop(columns=['Page count'], inplace=True)
microbiologyDataFiltered.drop(columns=['Link'], inplace=True)
microbiologyDataFiltered.drop(columns=['Source'], inplace=True)
microbiologyDataFiltered.drop(columns=['Volume'], inplace=True)
microbiologyDataFiltered.drop(columns=['Issue'], inplace=True)


# Replacing NaN Values
microbiologyDataFiltered['Cited by'] = microbiologyDataFiltered['Cited by'].fillna(0)

# Fixing types
microbiologyDataFiltered = microbiologyDataFiltered.astype(
    {
        'Year': int, 
        'Cited by': int
    }
)
print(microbiologyDataFiltered.dtypes)

# Filtering rows by 'Documnet Type'
microbiologyDataFiltered = microbiologyDataFiltered[microbiologyDataFiltered['Document Type'].isin(documentTypeSet)]


# Checking duplcates and dropping them
microbiologyDataCountOld = microbiologyDataFiltered.shape[0]

microbiologyDataFiltered = microbiologyDataFiltered.drop_duplicates(
    subset=duplicatesSubset,
    keep="first",
    inplace=False
)

microbiologyDataCountNew = microbiologyDataFiltered.shape[0]

print(f"Number of rows before deduplication '{microbiologyDataCountOld}'")
print(f"Number of rows after deduplication '{microbiologyDataCountNew}'")
print(f"Total removed '{microbiologyDataCountOld - microbiologyDataCountNew}' scientific papaers")

microbiologyDataFiltered.head(20)

## 1.4 Combining Datasets with scientific papers

In [ ]:
# Combining filtered epidemilogoy, immunology, infectiveDiseases and microbiology dataframes
combinedWorks = pd.concat(
    [
        epidemiologyDataFiltered,
        immunologyDataFiltered,
        infectiveDiseasesDataFiltered,
        microbiologyDataFiltered
        
    ], 
    ignore_index=True
)

combinedWorks = combinedWorks.drop_duplicates(
    subset=['Authors', 'Title', 'Year', 'Source title', 'Cited by', 'Document Type'],
    keep="first",
    inplace=False
)

combinedWorks.insert(0,'id_work',0)
combinedWorks['id_work'] = combinedWorks.reset_index().index + 1
combinedWorks.drop(columns=['Author'], inplace=True)

print(f"Combined dataset - Rows: {combinedWorks.shape[0]} - Cols: {combinedWorks.shape[1]}")
print(combinedWorks.dtypes)

combinedWorks.to_excel("data/combinedWorks.xlsx") 

combinedWorks.head(20)

# combinedWorks.groupby('Source title').size().reset_index(name='Num of work per Soruce title').sort_values(by='Num of work per Soruce title', ascending=False).head(50)


## 2. Research questions and objectives
### 2.1 Statistical data processing
#### 1. What is the number of works by each author? Use both whole and fractional counting. Who are the most productive scientists in the field of research and which department do they belong to?

In [38]:
# Separation of groups of authors into separate lines
workPerAuthor = combinedWorks.assign(Authors=combinedWorks['Authors'].str.split(',')).explode('Authors')
workPerAuthor.rename(columns={'Authors': 'Author'}, inplace=True)
workPerAuthor['Author'] = workPerAuthor['Author'].apply(lambda x: x.strip())

def f(x, short_name):
    if short_name in x:
        return short_name
    else:
        return x

for index, row in authorsData.iterrows():
    workPerAuthor['Author'] = workPerAuthor['Author'].apply(f, short_name = row['Short name'])

workPerAuthor.to_excel("data/workPerAuthor.xlsx") 

# Joined authorsData and workPerAuthor
authorsAndWorks = pd.merge(authorsData, workPerAuthor, left_on='Short name', right_on='Author', how='inner')

authorsAndWorks.to_excel("data/authorsAndWorks.xlsx") 

In [10]:
# Calacualte number of works per Author by workPerAuthor DataFrame and join with authorsData DataFrame

numOfWorksPerAuthor = workPerAuthor.groupby('Author').size().reset_index(name='Num of works per Author').sort_values(by='Num of works per Author', ascending=False)
    
numOfWorksPerAuthor.to_excel("data/workPerAuthorSorted.xlsx")

combineAuthorsAndNumOfWorks = pd.merge(numOfWorksPerAuthor, authorsData, left_on='Author', right_on='Short name', how='inner')
combineAuthorsAndNumOfWorks.to_excel("data/firstQuestion.xlsx")

# Show the results
# combineAuthorsAndNumOfWorks.sort_values(by='Num of works per Author', ascending=False).head(6)

# Show the results
# rezultat.head(80)

#### 2. What is the average number of co-authors per author ?

In [ ]:
# id_work, Authors, Title, Year, Source title, Cited by, Document Type, Research area

numOfAuthorsPerWork = workPerAuthor.groupby('id_work').size().reset_index(name='Num of authors per Work')
# numOfAuthorsPerWork.head(5)

numOfWorksPerAuthor = authorsAndWorks.groupby('Author').size().reset_index(name='Num of works per Author').sort_values(by='Num of works per Author', ascending=False)
# numOfWorksPerAuthor.head(70)

result1 = pd.merge(numOfAuthorsPerWork, authorsAndWorks, on='id_work', how='inner')
# result1.head(70)

result2 = pd.merge(result1, numOfWorksPerAuthor, on='Author', how='inner')
# result.head(70)

result2 = result2.groupby('Author')['Num of authors per Work'].sum().reset_index(name='Sum of Co Authors')
# result2.head(70)

result2 = pd.merge(result2, numOfWorksPerAuthor, on='Author', how='inner')
# result2.head(70)

result2['AVG CoAuthors per Author'] = (result2['Sum of Co Authors'] - result2['Num of works per Author']) / result2['Num of works per Author']
result2.sort_values(by='Sum of Co Authors', ascending=False).head(70)

#### 3. Based on the available data, determine the H-index of each scientist and compare it with the available H-index in the file authors.xlsx ?

In [ ]:
# The H-index matches us to a large extent in the given excel table authors.xlsx
authorWorksCitedBy = {}
authorHIndex = {}

for index, row in authorsData.iterrows():
    authorWorksCitedBy[row['Short name']] = []

for index, row in authorsAndWorks.iterrows():
    authorWorksCitedBy[row['Short name']].append(row['Cited by'])

for author in authorWorksCitedBy:
    authorWorksCitedBy[author].sort(reverse=True)

for author in authorWorksCitedBy:
    HIndex = 0
    
    for index, item in enumerate(authorWorksCitedBy[author]):
        if item >= index + 1:
            HIndex = index + 1
        else:
            break
    
    authorHIndex[author] = HIndex

authorsHIndex = pd.DataFrame(list(authorHIndex.items()), columns=['Short name', 'Calculate H index'])

combinedAuthorsAndHIndex = pd.merge(authorsHIndex, authorsData, on='Short name', how='inner')
combinedAuthorsAndHIndex.head(80)


#### 4. Which departments are the most productive in terms of scientific production and citations in journals based on the available data ?

In [ ]:
citedByPerAuthor = authorsAndWorks.groupby('Short name')['Cited by'].sum().reset_index(name='Sum of Cited by')
# citedByPerAuthor.head(80)

numOfWorksPerAuthor = authorsAndWorks.groupby('Short name').size().reset_index(name='Num of works per Author')
# numOfWorksPerAuthor.head(80)

result = pd.merge(citedByPerAuthor, numOfWorksPerAuthor, on='Short name', how='inner')
result = pd.merge(authorsData, result, on='Short name', how='inner')

result = result.groupby('Department').agg({'Sum of Cited by':'sum','Num of works per Author':'sum'})
result.sort_values(by='Sum of Cited by', ascending=False).head(10)

#### 5. In which years were the authors most productive at the faculty level and individual departments?

###### 5.1. By faculty level

In [ ]:
productivityPerYear = authorsAndWorks.groupby('Year').size().reset_index(name='Num of works')
productivityPerYear.sort_values(by='Num of works', ascending=False).head(100)

###### 5.2. By department level

In [ ]:
productivityPerYearAndDeprtment = authorsAndWorks.groupby(['Year', 'Department']).size().reset_index(name='Num of works')
productivityPerYearAndDeprtment.sort_values(by=['Year', 'Num of works'], ascending=[False, False]).head(50)

#### 6. In which magazines is the most published on average ?

In [ ]:
numOfWorlsPerArticle = authorsAndWorks.groupby(['Source title']).size().reset_index(name='Num of works')
numOfWorlsPerArticle.sort_values(by='Num of works', ascending=False).head(10)

#### 7. Are there differences between departments in terms of volume and frequency of publication in journals ?
From task number 6, it can be seen that the journal 'Archives of Biological Sciences' has the most publications, and in the picture below we can see that there is not an excessively large oscillation in terms of publications by journals by different departments.

In [ ]:
numOfWorksPerArticleAndDepartment = authorsAndWorks.groupby(['Source title', 'Year','Department']).size().reset_index(name='Num of works')
# numOfWorksPerArticleAndDepartment.sort_values(by='Year', ascending=False).head(70)

grafik = sns.barplot(
    data=productivityPerYearAndDeprtment.sort_values(by=['Year'], ascending=[False]).head(36),
    x='Year', 
    y='Num of works', 
    hue='Department'
)
sns.move_legend(grafik, "upper left", bbox_to_anchor=(1, 1))
plt.show()

#### 8. Is there a difference in the average number of authors per paper in journals per department ?

In [ ]:
numOfAuthorsPerDepartment = authorsData.groupby('Department').size().reset_index(name='Num of authors')
# numOfAuthorsPerDepartment.head(5)

numOfWorksPerDepratment = combineAuthorsAndNumOfWorks.groupby('Department')['Num of works per Author'].sum().reset_index(name='Num of works')
# numOfWorksPerDepratment.head(5)

result = pd.merge(numOfAuthorsPerDepartment, numOfWorksPerDepratment, on='Department', how='inner')

result['Author per work'] = result['Num of authors'] / result['Num of works']
result['AVG Author per work'] = result['Author per work'].mean()
result.head(5)

#### 9. What is the ratio of the number of co-authors from the faculty in relation to the number of authors outside the faculty by departments and at the level of the entire faculty ?

In [ ]:
numOfDomesticAuthorsPerWork = authorsAndWorks.groupby(['id_work', 'Title']).size().reset_index(name='Num of domestic authors')
# numOfDomesticAuthorsPerWork.head(20)

foreginAuthorAndWorks = pd.merge(workPerAuthor, authorsData, left_on='Author', right_on='Short name', how='left')
filtered_df = foreginAuthorAndWorks[foreginAuthorAndWorks['Author'].isin(authorsData['Short name'])]
foreginAuthorAndWorks = foreginAuthorAndWorks.drop(filtered_df.index)
# foreginAuthorAndWorks.head(40)

numOfForeginAuthorsPerWork = foreginAuthorAndWorks.groupby(['id_work', 'Title']).size().reset_index(name='Num of foregin authors')
# numOfForeginAuthorsPerWork.head(40)

numOfForeginAndDomesitAuthorsPerWork = pd.merge(numOfDomesticAuthorsPerWork, numOfForeginAuthorsPerWork, on=['id_work', 'Title'])
# numOfForeginAndDomesitAuthorsPerWork.head(40)

mergedDomesticAuhorsWorksAndNumOfCoAuthors = pd.merge(authorsAndWorks, numOfForeginAndDomesitAuthorsPerWork, on='id_work', how='inner')
# result9.head(25)

noDuplicatedWorks = mergedDomesticAuhorsWorksAndNumOfCoAuthors[~mergedDomesticAuhorsWorksAndNumOfCoAuthors.duplicated(['id_work', 'Department'], keep=False)]
# df_no_duplicates.head(10)

groupbyDepartment = noDuplicatedWorks.groupby('Department')[['Num of domestic authors', 'Num of foregin authors']].sum()
groupbyDepartment['The ratio of domestic and foregin authors'] = groupbyDepartment['Num of domestic authors'] / groupbyDepartment['Num of foregin authors']

groupbyDepartment.head(5)


In [ ]:
foreginAuthorAndWorks.groupby('Author').size().reset_index(name='Num of works per Author').sort_values(by='Num of works per Author', ascending=False).head(20)


### 2.2 Basic characterization of modeled networks

In [ ]:
# TODO: Export DataFrame 'authorsAndWorks' via Pickle to a file and load it here
inputCleanData = authorsAndWorks
labels = set(inputCleanData['Short name'].unique())
print(f"There are { len(labels) } different authors")

In [ ]:
reducedAuthorsAndWorks = inputCleanData[['id_work', 'Author']]

combinations_df = pd.merge(reducedAuthorsAndWorks, reducedAuthorsAndWorks, on='id_work')
# combinations_df.head(80)

# Filtering pairs of authors that are not the same (to avoid pairs of authors with themselves)
filtered_combinations_df = combinations_df[combinations_df['Author_x'] != combinations_df['Author_y']]

# Creating a new DataFrame with columns 'id_work', 'author' and 'co-author'
result_df = filtered_combinations_df[['id_work', 'Author_x', 'Author_y']].rename(columns={'Author_x': 'Author', 'Author_y': 'Co-Author'})
result_df.head(-1)

In [ ]:
G = nx.Graph()
G.add_nodes_from(labels)

for _, idWork, author, coAuthor in result_df.itertuples():
    if (author, coAuthor) in G.edges:
        G.edges[author, coAuthor]['weight'] += 1
    else:
        G.add_edge(author, coAuthor, weight=1)

print(G.edges.data("weight"))

nx.write_gml(G, "models/mreza_jedan.gml")

In [ ]:
H = G.to_undirected()

print("Number of connected components: " , nx.number_connected_components(H))

connected_componnets = sorted(nx.connected_components(H), key=len, reverse=True)
print("Size of connected components:")
for x in connected_componnets:
    print(len(x) , "  ")
    
print( "Average clustering coefficient:" , nx.average_clustering(H), ", Global clustering coefficient: " , nx.transitivity(H) )

In [ ]:
# Local Degree of clustering
author, clustering_coef = zip(*nx.clustering(G, weight = "weight").items())
no_zero = [(auth, cc)  for auth, cc in zip(author, clustering_coef) if cc > 0]

df = pd.DataFrame(no_zero, columns = ["Author", "Cc"])
df.sort_values('Cc', inplace = True)

max_local_degree_of_clustering = max(clustering_coef)
average_degree_of_clustering = nx.average_clustering(G)

print(f"Max local cc: {max_local_degree_of_clustering}")
print(f"Average cc: {average_degree_of_clustering}")
print("Local degree of clustering which is not a zero:")
print(df)

# ax = df.plot.scatter(x='Author', y='Cc')
# ax.set_xscale("log")
# ax.set_yscale("log");

plt.hist(clustering_coef, bins=10, edgecolor='black', color = "green")

# Set axis titles and labels
plt.title('Local Clustering Coefficient Histogram')
plt.xlabel('Local Clustering Coefficient')
plt.ylabel('Frequency')
plt.savefig('images/local_cc_main_network.png')

# Displaying a histogram
plt.show()


In [26]:
def plot_deg_frequency(G, weighted = False, xscale = "log", yscale = "log"):

    if weighted:
        degrees = G.degree(weight="weight")
    else:
        degrees = G.degree()
        
    _, deg_list = zip(*degrees)
    deg_counts = Counter(deg_list)        
    print(deg_counts)
    x, y = zip(*deg_counts.items())                                                      

    plt.figure(1)   

    # prep axes   
    if weighted:
        plt.xlabel('weighted degree')  
    else:
        plt.xlabel('degree')                                                                                                             
    plt.xscale(xscale)                                                                                                                
    plt.xlim(1, max(x))  

    plt.ylabel('frequency')                                                                                                          
    plt.yscale(yscale)                                                                                                                
    plt.ylim(1, max(y))                                                                                                             
                                                                                                                                                                                                    
    plt.scatter(x, y, marker='.')                                                                                                    
    plt.show()


In [ ]:
n = G.number_of_nodes()
m = G.number_of_edges()

print(f"Nodes: {n}")
print(f"Edges: {m}")

Gnm = nx.gnm_random_graph(n, m) 

# plot_deg_frequency(Gnm, xscale = 'linear', yscale = 'linear')
plot_deg_frequency(Gnm, xscale = 'log', yscale = 'log')

In [ ]:
# Erdős–Rényi network N = 58 (number of nodes - authors)

p = (2 * float(m)) / (n * ( n - 1))
print(p)

er_network = nx.erdos_renyi_graph(n,p)

delta_m = m - er_network.number_of_edges()
print(f"The number of nodes of the original network minus the number of nodes in the ER network amounts to {delta_m}, what is the deviation from {abs(float(delta_m)) * 100 / m}%")

# plot_deg_frequency(er_network, xscale = 'linear', yscale = 'linear')
plot_deg_frequency(er_network, xscale = 'log', yscale = 'log')

# Local clustering degree
author, clustering_coef = zip(*nx.clustering(er_network, weight = "weight").items())
no_zero = [(auth, cc)  for auth, cc in zip(author, clustering_coef) if cc > 0]

df = pd.DataFrame(no_zero, columns = ["Author", "Cc"])
df.sort_values('Cc', inplace = True)

max_local_degree_of_clustering = max(clustering_coef)
prosecni_stepen_klasterisanja = nx.average_clustering(er_network)

print(f"Max local cc Erods-Reny network: {max_local_degree_of_clustering}")
print(f"Average cc Erods-Reny network: {prosecni_stepen_klasterisanja}")
print("Local clustering degree which isn't zero Erods-Reny network:")
print(df)

plt.hist(clustering_coef, bins=10, edgecolor='black', color = "red")

# Axis title and label settings
plt.title('Local Clustering Coefficient Histogram')
plt.xlabel('Local Clustering Coefficient')
plt.ylabel('Frequency')
plt.savefig('images/local_cc_er_network.png')

# Show histogram
plt.show()

In [ ]:
# Random generated ScaleFree network
scale_free_network = nx.scale_free_graph(n)

# Create weighted graph from scale_free_network
Q = nx.DiGraph()
for u,v in scale_free_network.edges():
    if Q.has_edge(u,v):
        Q[u][v]['weight'] += 1
    else:
        Q.add_edge(u, v, weight=1)
        
# Local clustering degree
author, clustering_coef = zip(*nx.clustering(Q, weight = "weight").items())
nenula = [(auth, cc)  for auth, cc in zip(author, clustering_coef) if cc > 0]

df = pd.DataFrame(nenula, columns = ["Author", "Cc"])
df.sort_values('Cc', inplace = True)

max_local_degree_of_clustering = max(clustering_coef)

average_degree_of_clustering = nx.average_clustering(Q)

print(f"Max local cc Random scale-free network: {max_local_degree_of_clustering}")
print(f"Average cc Random scale-free network: {average_degree_of_clustering}")
print("Local clustering degree which isn't zero Random scale-free network:")
print(df)

plt.hist(clustering_coef, bins=10, edgecolor='black')

plt.title('Local Clustering Coefficient Histogram')
plt.xlabel('Local Clustering Coefficient')
plt.ylabel('Frequency')

plt.savefig('images/local_cc_random_scale_free_network.png')

plt.show()

In [ ]:
# Assortativeness 
# Assortativeness based on the unweighted degree of the node
r1 = nx.degree_assortativity_coefficient(G)
print(f"Coefficient of assortativity based on the unweighted degree of the node: {r1}")

# Assortability based on the degree of difficulty of the node
r2 = nx.degree_assortativity_coefficient(G, weight='weight')
print(f"Coefficient of assortativity based on the weight degree of the node: {r2}")



In [ ]:
# Rich club phenomenon; Havel-Hakimi algorithm
degrees = G.degree()
_, deg_list = zip(*degrees)
print(deg_list)

havel_hakimi_network = nx.havel_hakimi_graph(deg_list)

# nx.draw(havel_hakimi_network)
nx.write_gml(havel_hakimi_network, "models/havel_hakimi.gml")

In [ ]:
rich_club_coeffs = nx.rich_club_coefficient(G.to_undirected())
rich_club_coeffs_havel_hakimi = nx.rich_club_coefficient(havel_hakimi_network)

if rich_club_coeffs:
    degrees = list(rich_club_coeffs.keys())
    coefficients = list(rich_club_coeffs.values())
    print(degrees)
    print(coefficients)
    plt.plot(degrees, coefficients, marker='o')
    plt.xlabel('Degree (k)')
    plt.ylabel('Rich-club Coefficient')
    plt.title('Rich-club Phenomenon')
    plt.show()
else:
    print("The network does not contain enough nodes to analyze the rich-club phenomenon.")

if rich_club_coeffs_havel_hakimi:
    degrees = list(rich_club_coeffs_havel_hakimi.keys())
    coefficients = list(rich_club_coeffs_havel_hakimi.values())
    print(degrees)
    print(coefficients)
    plt.plot(degrees, coefficients, marker='o', color='red')
    plt.xlabel('Degree (k)')
    plt.ylabel('Rich-club Coefficient')
    plt.title('Rich-club Phenomenon Havel-Hakimi')
    plt.show()
else:
    print("The network does not contain enough nodes to analyze the rich-club phenomenon.")


In [ ]:
# Power law distribution
degrees = dict(G.degree())
degree_values = list(degrees.values())

plt.hist(degree_values, bins=30, log=True)
plt.title('Degree Distribution Histogram')
plt.xlabel('Number of nodes')
plt.ylabel('Number of nodes (log scale)')
plt.show()

In [ ]:
# Assortative mixing
def plot_deg_corr(g, xscale = "linear", yscale = "linear"):
    first = []
    second = []
    for i, k in g.edges():
        first.append(g.degree(i))
        first.append(g.degree(k))
        second.append(g.degree(k))
        second.append(g.degree(i))

    plt.figure(1)   

    plt.xlabel('degree')                                                                                                             
    plt.xscale(xscale)                                                                                                                
    plt.xlim(1, max(first)) 

    plt.ylabel('degree')                                                                                                          
    plt.yscale(yscale)                                                                                                                
    plt.ylim(1, max(second))                                                                                                             

    plt.scatter(first, second, marker='.')                                                                                                    
    plt.show()

plot_deg_corr(G)

assortativity = nx.degree_assortativity_coefficient(G)
print(f"Mixing assortativeness coefficient: {assortativity}")

### 2.3 Analysis of centrality measures

In [ ]:
def calculate_centralities(G):

    DC_dict = nx.degree_centrality(G)
    CC_dict = nx.closeness_centrality(G)
    BC_dict = nx.betweenness_centrality(G)
    EVC_dict = nx.eigenvector_centrality(G)

    df1 = pd.DataFrame.from_dict(DC_dict, orient='index', columns=['DC'])
    df2 = pd.DataFrame.from_dict(CC_dict, orient='index', columns=['CC'])
    df3 = pd.DataFrame.from_dict(BC_dict, orient='index', columns=['BC'])
    df4 = pd.DataFrame.from_dict(EVC_dict, orient='index', columns=['EVC'])
    df = pd.concat([df1, df2, df3, df4], axis=1)
    return df

calcCentralitiesDf = calculate_centralities(G)

print(calcCentralitiesDf)
print(calcCentralitiesDf.dtypes)

In [ ]:
def calculate_centralities_v2(G):

    DC_dict = nx.degree_centrality(G)
    CC_dict = nx.closeness_centrality(G)
    BC_dict = nx.betweenness_centrality(G)
    EVC_dict = nx.eigenvector_centrality(G)
        
    df1 = pd.DataFrame(list(DC_dict.items()), columns=['Author', 'Degree centrality'])
    df2 = pd.DataFrame(list(CC_dict.items()), columns=['Author', 'Closeness centrality'])
    df3 = pd.DataFrame(list(BC_dict.items()), columns=['Author', 'Betweenness centrality'])
    df4 = pd.DataFrame(list(EVC_dict.items()), columns=['Author', 'Eigenvector centrality'])
    df = df1.merge(df2,on='Author').merge(df3,on='Author').merge(df4,on='Author')
    return df


centralitiesDf = calculate_centralities_v2(G)

centralitiesDf = centralitiesDf.merge(authorsData, left_on='Author', right_on='Kratko ime', how='inner')
centralitiesDf.sort_values(by='Degree centrality', ascending=False).head(-1)

In [ ]:
centralitiesDf.sort_values(by='Eigenvector centrality', ascending=False).head(-1)

In [ ]:
# Heuristics
labels = ['DC', 'CC', 'BC', 'EVC']
cross_correlation_matrix = pd.DataFrame(columns = ['DC', 'CC', 'BC', 'EVC'], index = ['DC', 'CC', 'BC', 'EVC'])
p_val_matrix = pd.DataFrame(columns = ['DC', 'CC', 'BC', 'EVC'], index = ['DC', 'CC', 'BC', 'EVC'])

for ind in labels:
    for col in labels:
        cross_correlation_matrix[ind][col], p_val_matrix[ind][col] = stats.kendalltau(calcCentralitiesDf[ind], calcCentralitiesDf[col])

print(cross_correlation_matrix)
print(p_val_matrix)

In [ ]:
df_ = calculate_centralities(G)
for metric in labels:
    df_[f"{metric}_rank"] = df_[f"{metric}"].rank(ascending=False) 
    
df_['composite_rank'] = df_['DC_rank'] * df_['CC_rank'] * df_['BC_rank'] * df_['EVC_rank']

df_.sort_values(['composite_rank'], ascending = True)

#  df_.sort_values(['CC'], ascending = False, inplace=True)

df_.head(-1)

## 3. Commune detection by spectral clustering

In [ ]:
# Grivan-Newman Method

# grivanNewman = nx.community.girvan_newman(G)
# print(grivanNewman)

# k = 10
# limited = itertools.takewhile(lambda c: len(c) <= k, grivanNewman)
# for communities in limited:
# print(tuple(sorted(c) for c in communities))

In [ ]:
grivanNewman = nx.community.girvan_newman(G)

k = 6  # set the desired number of communes
communities = list(sorted(c) for c in next(grivanNewman) if len(c) >= k)

# G = nx.path_graph(10)
communities = list(nx.community.girvan_newman(G))

# building initial dict of node_id to each possible subset:
node_id = 0
init_node2community_dict = {node_id: communities[0][0].union(communities[0][1])}
for comm in communities:
    for subset in list(comm):
        if subset not in init_node2community_dict.values():
            node_id += 1
            init_node2community_dict[node_id] = subset

# turning this dictionary to the desired format in @mdml's answer
node_id_to_children = {e: [] for e in init_node2community_dict.keys()}
for node_id1, node_id2 in combinations(init_node2community_dict.keys(), 2):
    for node_id_parent, group in init_node2community_dict.items():
        if len(init_node2community_dict[node_id1].intersection(init_node2community_dict[node_id2])) == 0 and group == init_node2community_dict[node_id1].union(init_node2community_dict[node_id2]):
            node_id_to_children[node_id_parent].append(node_id1)
            node_id_to_children[node_id_parent].append(node_id2)

# also recording node_labels dict for the correct label for dendrogram leaves
node_labels = dict()
for node_id, group in init_node2community_dict.items():
    if len(group) == 1:
        node_labels[node_id] = list(group)[0]
    else:
        node_labels[node_id] = ''

# also needing a subset to rank dict to later know within all k-length merges which came first
subset_rank_dict = dict()
rank = 0
for e in communities[::-1]:
    for p in list(e):
        if tuple(p) not in subset_rank_dict:
            subset_rank_dict[tuple(sorted(p))] = rank
            rank += 1
subset_rank_dict[tuple(sorted(chain.from_iterable(communities[-1])))] = rank

# function to get a merge height so that it is unique (probably not that efficient)
def get_merge_height(sub):
    sub_tuple = tuple(sorted([node_labels[i] for i in sub]))
    n = len(sub_tuple)
    other_same_len_merges = {k: v for k, v in subset_rank_dict.items() if len(k) == n}
    min_rank, max_rank = min(other_same_len_merges.values()), max(other_same_len_merges.values())
    range = (max_rank-min_rank) if max_rank > min_rank else 1
    return float(len(sub)) + 0.8 * (subset_rank_dict[sub_tuple] - min_rank) / range

# finally using @mdml's magic, slightly modified:
G           = nx.DiGraph(node_id_to_children)
nodes       = G.nodes()
leaves      = set( n for n in nodes if G.out_degree(n) == 0 )
inner_nodes = [ n for n in nodes if G.out_degree(n) > 0 ]

# Compute the size of each subtree
subtree = dict( (n, [n]) for n in leaves )
for u in inner_nodes:
    children = set()
    node_list = list(node_id_to_children[u])
    while len(node_list) > 0:
        v = node_list.pop(0)
        children.add( v )
        node_list += node_id_to_children[v]
    subtree[u] = sorted(children & leaves)

inner_nodes.sort(key=lambda n: len(subtree[n])) # <-- order inner nodes ascending by subtree size, root is last

# Construct the linkage matrix
leaves = sorted(leaves)
index  = dict( (tuple([n]), i) for i, n in enumerate(leaves) )
Z = []
k = len(leaves)
for i, n in enumerate(inner_nodes):
    children = node_id_to_children[n]
    x = children[0]
    for y in children[1:]:
        z = tuple(sorted(subtree[x] + subtree[y]))
        i, j = index[tuple(sorted(subtree[x]))], index[tuple(sorted(subtree[y]))]
        Z.append([i, j, get_merge_height(subtree[n]), len(z)]) # <-- float is required by the dendrogram function
        index[z] = k
        subtree[z] = list(z)
        x = z
        k += 1

# Dendrogram
plt.figure()
dendrogram(Z, labels=[node_labels[node_id] for node_id in leaves])

In [ ]:
# In gephi, we put 2.5 for Community Detection, for a case similar to a dendogram
# Importing the required library
from sklearn.cluster import SpectralClustering

# Defining the number of clusters
n_clusters = 3

# Calculation of spectral clustering
clustering = SpectralClustering(n_clusters=n_clusters, assign_labels="discretize", affinity="precomputed").fit(nx.adjacency_matrix(G).toarray())

colors = clustering.labels_
c_string = []
for c in colors:
    c_string.append(str(c))

spectral_clust_graph = nx.Graph()
for c, label in zip(c_string, G.nodes()):
    spectral_clust_graph.add_node(label, color=c)

for edge in G.edges(data=True):
    #print(edge)
    spectral_clust_graph.add_edge(edge[0], edge[1], weight=edge[2]['weight'])

nx.write_pajek(spectral_clust_graph, "models/spectral4.net")

csizes = np.zeros(n_clusters)
for c in colors:
    csizes[int(c)] += 1

print(f"Division into {n_clusters}: the component sizes are {csizes}")